In [ ]:
import numpy as np
from astropy.io import fits
from astropy.io.fits import HDUList
from astropy.wcs import WCS
from numpy import arcsinh
import matplotlib.pyplot as plt
from astropy.visualization import astropy_mpl_style
plt.style.use(astropy_mpl_style)
from reproject import reproject_interp
from aplpy.rgb import make_rgb_cube

In [ ]:
%ls -l

In [ ]:
def load_file(filename):
    fits_data = fits.open(filename)
    hdu = fits_data[0]
    header = hdu.header
    data = hdu.data
    
    return data, header, WCS(header), hdu

def world2pix(ra, dec, wcs):
    coords = np.array([[ra, dec]])
    location = wcs.wcs_world2pix(coords, 0, ra_dec_order=True)
    return location[0][0], location[0][1]

In [ ]:
R_IDX = 0
G_IDX = 1
B_IDX = 2

def scale_rgb(r, g, b, sigma=6, min=0, max=10000, gains=[0.9,1.1,1.8], gamma=0.1):
    r = r.copy()
    g = g.copy()
    b = b.copy()

    slope = 255 / arcsinh((max - min)/sigma)
    max_x, max_y = r.shape

    mean = (r + g + b)/3
    mean[mean < min] = 0
    r[mean == 0] = 0
    g[mean == 0] = 0
    b[mean == 0] = 0
    
    scale = slope * arcsinh((mean - min) / sigma) / mean

    r = (r * scale).astype(int)
    g = (g * scale).astype(int)
    b = (b * scale).astype(int)
    
    r = (r * gains[R_IDX]).astype(int)
    g = (g * gains[G_IDX]).astype(int)
    b = (b * gains[B_IDX]).astype(int)
    
    r += (gamma * (r - g)).astype(int)
    b += (gamma * (b - g)).astype(int)

    r[r < 0] = 0
    r[r > 255] = 255
    g[g < 0] = 0
    g[g > 255] = 255
    b[b < 0] = 0
    b[b > 255] = 255
            
    return r, g, b

def cutout(data, wcs, ra, dec, x_size=100, y_size=100):
    x_centre, y_centre = world2pix(ra, dec, wcs)
    x_top = int(round(x_centre) - x_size/2)
    y_top = int(round(y_centre) - y_size/2)
    x_bottom = x_top + x_size
    y_bottom = y_top + y_size
    
    return data[y_top:y_bottom, x_top:x_bottom]

In [ ]:
u_data, u_header, u_wcs, u_hdu = load_file('frame-u-002141-3-0076.fits')
g_data, g_header, g_wcs, g_hdu = load_file('frame-g-002141-3-0076.fits')
r_data, r_header, r_wcs, r_hdu = load_file('frame-r-002141-3-0076.fits')
i_data, i_header, i_wcs, i_hdu = load_file('frame-i-002141-3-0076.fits')
z_data, z_header, z_wcs, z_hdu = load_file('frame-z-002141-3-0076.fits')

In [ ]:
min((g_data.min(), r_data.min(), i_data.min()))

In [ ]:
max((g_data.max(), r_data.max(), i_data.max()))

In [ ]:
hdulist = HDUList([g_hdu, r_hdu, i_hdu])

In [ ]:
i_hdu.header

In [ ]:
data_array, footprint = reproject_interp(g_hdu, r_header)

In [ ]:
g_data

In [ ]:
plt.imshow(g_data)

In [ ]:
data_array

In [ ]:
plt.imshow(data_array)

In [ ]:
ra, dec = (153.937453927558,-0.371692014858263)
red_cutout = cutout(i_data, i_wcs, ra, dec, 50, 50)
green_cutout = cutout(r_data, r_wcs, ra, dec, 50, 50)
blue_cutout = cutout(g_data, g_wcs, ra, dec, 50, 50)

In [ ]:
r, g, b = scale_rgb(red_cutout, green_cutout, blue_cutout)

In [ ]:
grey = np.power((0.2126 * np.power(r, 2.2) + 0.7152 * np.power(g, 2.2) + 0.0722 + np.power(b, 2.2)), 1/2.2) # (0.2126×Red2.2 + 0.7152×Green2.2 + 0.0722×Blue2.2

In [ ]:
grey *= 255/np.max(grey)

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(grey.T, cmap='Reds')
plt.colorbar()

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(scaled_data[2], cmap='gray')
plt.colorbar()

In [ ]:
ra = 153.925821207403
dec = -0.348890142024622

In [ ]:
print(world2pix(ra, dec, g_wcs))
print(world2pix(ra, dec, i_wcs))
print(world2pix(ra, dec, r_wcs))
print(world2pix(ra, dec, u_wcs))
print(world2pix(ra, dec, z_wcs))


In [ ]:
ra_centre = location[0][0]
dec_centre = location[0][1]
size = 40

In [ ]:
print(ra_centre, dec_centre)

In [ ]:
ra_top = int(ra_centre - size/2)
dec_top = int(dec_centre - size/2)
ra_bottom = ra_top + size
dec_bottom = dec_top + size

In [ ]:
fits_data[0].data.shape

In [ ]:
image_data = fits_data[0].data[dec_top:dec_bottom, ra_top:ra_bottom]

In [ ]:
image_data.size

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(image_data, cmap='gray')
plt.colorbar()